In [20]:
import os
import torch

from dotenv import load_dotenv

from pymongo import MongoClient
from langchain_mongodb import MongoDBAtlasVectorSearch

from langchain_community.document_loaders.pdf import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings

from transformers import BitsAndBytesConfig, AutoModelForCausalLM, AutoTokenizer, pipeline
from langchain_huggingface import HuggingFacePipeline

from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

# Variables

In [21]:
load_dotenv()

True

In [22]:
file = os.getenv("FILE")
token = os.getenv("TOKEN")
model_id = os.getenv("MODEL_ID")
model_name = os.getenv("MODEL_NAME")
mongo_url = os.getenv("MONGO_URL")
mongo_db = os.getenv("MONGO_DB")
mongo_collection = os.getenv("MONGO_COLLECTION")

# PDFLoader

In [23]:
loader = PyPDFLoader(file)
files = loader.load()

In [24]:
id = "45657525-1a48-4c82-88aa-a9f9037b2d4b"

In [25]:
for file in files:
    file.metadata['id'] = str(id)

# Recursive TextSplit

In [26]:
recur_split = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=150
)

In [27]:
documents = recur_split.split_documents(files)

# Embeddings (HuggingFace)

In [28]:
embeddings = HuggingFaceEmbeddings(
    model_name=model_name, 
    model_kwargs={'device': 'cuda:0'}
)

# Vector Store

In [29]:
client = MongoClient(mongo_url)
db = client[mongo_db]
collection = db[mongo_collection]

In [30]:
vectordb = MongoDBAtlasVectorSearch.from_documents( 
    documents=documents, 
    embedding=embeddings, 
    collection=collection,
    index_name="vector_index"
)

_id or id key found in metadata. Please pop from each dict and input as separate list.Retrieving methods will include the same id as '_id' in metadata.


# Retriever

In [31]:
retriever = vectordb.as_retriever(
    search_type='similarity', 
    search_kwargs = {
        "k": 5,
        'fetch_k': 30,
        'pre_filter': {'id': str(id)}
    }
)

# LLM (HuggingFace)

In [32]:
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True, 
    bnb_4bit_use_double_quant=True, 
    bnb_4bit_quant_type="nf4", 
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    token=token,
    quantization_config=quantization_config    
)

tokenizer = AutoTokenizer.from_pretrained(
    model_id,
    token=token    
)

pipe = pipeline(
    model=model, 
    tokenizer=tokenizer, 
    task="text-generation",
    return_full_text=False
)

llm = HuggingFacePipeline(
    pipeline=pipe,
    model_kwargs={
        "temperature": 0.0,        
        "do_sample": False        
    }
)

Loading checkpoint shards: 100%|██████████| 2/2 [00:04<00:00,  2.28s/it]
Device set to use cuda:0


# Prompt

In [33]:
token_start, token_end = "<|begin_of_text|><|start_header_id|>system<|end_header_id|>", "<|eot_id|><|start_header_id|>assistant<|end_header_id|>"

template = """
Você é um assistente de extração de dados educacionais.  
Sua tarefa é localizar **exatamente** os Códigos e Descrições **que já estão presentes no contexto abaixo** de acordo com os dados fornecidos.  

{context}

REGRAS IMPORTANTES:
- Você **NÃO** pode inventar nenhum conteúdo que não esteja literalmente no contexto acima.
- Você deve retornar **apenas os códigos e descrições exatos e completos** que aparecem no contexto.
- Você pode retornar um ou mais códigos e descrições, desde que esteja de acordo com os dados fornecidos.
- **Não corte partes das descrições.**
- **Não relacione o conteúdo com códigos errados.**

FORMATO OBRIGATÓRIO DE SAÍDA (copiar literalmente do contexto):
Código: <código exato>  
Descrição: <descrição completa>

---

Pergunta do usuário:  
{input}
"""

prompt = ChatPromptTemplate.from_template(token_start + template + token_end)

# Chain

In [34]:
documents_chain = create_stuff_documents_chain(
    llm, 
    prompt
)

In [35]:
retriver_chain = create_retrieval_chain(
    retriever=retriever, 
    combine_docs_chain=documents_chain
)

# Question

In [36]:
question = "**Ano Letivo** <9 ano>, **Disciplina** <Matemática>, sendo o **Conteúdo** <Porcentagem> e o **Objetivo da Aula** é <Calcular Taxa de Juros>."

# Answer

In [37]:
retriver_chain.invoke({"input": question})

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


{'input': '**Ano Letivo** <9 ano>, **Disciplina** <Matemática>, sendo o **Conteúdo** <Porcentagem> e o **Objetivo da Aula** é <Calcular Taxa de Juros>.',
 'context': [Document(id='689e8d5e3357769e8f1121a7', metadata={'_id': '689e8d5e3357769e8f1121a7', 'producer': 'PyFPDF 1.7.2 http://pyfpdf.googlecode.com/', 'creator': 'PyPDF', 'creationdate': 'D:20250429004401', 'source': '../pdf/matematica.pdf', 'total_pages': 3, 'page': 2, 'page_label': '3', 'id': '45657525-1a48-4c82-88aa-a9f9037b2d4b'}, page_content='vezes propositadamente, erros de leitura, como escalas inapropriadas, legendas não explicitadas\ncorretamente, omissão de informações importantes (fontes e datas), entre outros.\nCódigo: EF09MA21\nDescrição: Escolher e construir o gráfico mais adequado (colunas, setores, linhas), com ou sem uso de\nplanilhas eletrônicas, para apresentar um determinado conjunto de dados, destacando aspectos como as\nmedidas de tendência central.\nCódigo: EF09MA22\nDescrição: Planejar e executar pesquisa